# ML Course 2024 |  Medical Expenses Prediction Challenge

This notebook should serve as a starting point to work on the project. Please read the project description first.

In [ ]:
# Just so that you don't have to restart the notebook with every change.
%load_ext autoreload
%autoreload 2 

# Set team ID
Important: set your Team ID here. You can find it in CMS.

In [62]:
team_id = "18"  # put your team id here

# [Colab only] Connect to your Google Drive

In [63]:
# from google.colab import drive
# drive.mount('/content/drive')

In [64]:
# %cd "/content/drive/MyDrive/path/to/your/project"

# Imports

[Colab only] Note: if you need to install any packages, run a code cell with content `!pip install packagename`

In [65]:
from utils import config
from utils.config import PAPER_STYLE

In [66]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics as skm
from prettytable import PrettyTable
from scipy import stats
from scipy.stats import norm, skew  # for some statistics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFECV, SelectKBest, VarianceThreshold, chi2, f_classif, mutual_info_classif
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, RidgeClassifier, SGDClassifier
from sklearn.model_selection import (
    GridSearchCV,
    ShuffleSplit,
    StratifiedKFold,
    TunedThresholdClassifierCV,
    train_test_split,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    PolynomialFeatures,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    quantile_transform,
)
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR, NuSVC, NuSVR
from umap import UMAP
from xgboost import XGBClassifier

from utils import experiments, helpers
from utils import metrics as my_metrics
from utils import plots, scorers
from utils import statistical as st
from utils import tuning
from utils.param_grids import (
    choose_param_grid,
    combine_param_grids,
    construct_param_grids_list,
    make_smaller_param_grid,
)
from utils.transformers import PolynomialColumnTransformer

# Use cuML to use GPU-accelerated models
USE_CUML = False

# Run experiments
RUN_EXPERIMENTS = False
EXECUTE_CORRELATION_ANALYSIS = False
CORRELATION_THRESHOLD = 0.9

# True if no plots should be displayed
NO_PLOTS = False

if config.CUML_INSTALLED and USE_CUML:
    from cuml import LogisticRegression, MBSGDClassifier
    from cuml.common.device_selection import get_global_device_type, set_global_device_type
    from cuml.svm import SVC, SVR

    set_global_device_type("gpu")

    print("cuML's default execution device:", get_global_device_type())

# Constants that define the classification and regression targets
CLF_TARGET = "UTILIZATION"
REG_TARGET = "TOT_MED_EXP"

# Dedicate a fraction of the data for testing (validation is taken care of by CV)
TEST_SIZE = 0.2

# Define a RANDOM_STATE to make outputs deterministic
RANDOM_STATE = 42
helpers.seed_everything(RANDOM_STATE)

# **Data Analysis & Preprocessing**

## Load Data

In a first step, we load the provided training data from the csv file

In [ ]:
df_train = pd.read_csv("data/train.csv")
df_train.drop(columns=[REG_TARGET], inplace=True) # drop the regression target

print("The loaded dataset has {} rows and {} columns".format(df_train.shape[0], df_train.shape[1]))

In [ ]:
df_train.head()

In [ ]:
# Handling missing values
total_missing_values = df_train.isnull().sum().sum()
print(f"Total number of missing values: {total_missing_values}")

## Data exploration

In [ ]:
print(
    helpers.describe_cols(
        df_train,
        dummy_is_categorical=False,
        consecutive_sequences_are_categorical=False,
        low_unique_int_values_are_categorical=False,
    )
)

In [ ]:
categorical_cols, numerical_cols = helpers.categorical_and_numerical_columns(
    df_train,
    consecutive_sequences_are_categorical=False,
    low_unique_int_values_are_categorical=False,
)

# One-Hot encoding for categorical columns (with non-numeric values only)
categorical_encoder, df_train = helpers.handle_categorical_cols(
    df_train, categorical_cols, return_only_encoded=False
)
df_train = df_train.copy()

print(f"Numerical ({len(numerical_cols)}) ", numerical_cols)
print(f"Categorical ({len(categorical_cols)}): ", categorical_cols)
print(f"Number of columns after one-hot encoding: {df_train.shape[1]}")

In [ ]:
# RACE_White
# If 1.0, the person is white, otherwise not
# UTILIZATION_LOW
# If 1.0, the person has low utilization, otherwise high
df_train.rename(columns={"RACE_White": "RACE"}, inplace=True)
df_train.rename(columns={f"{CLF_TARGET}_LOW": f"{CLF_TARGET}"}, inplace=True)

CLASS_MAPPING = {1: "LOW", 0: "HIGH"}

print(
    helpers.describe_cols(
        df_train,
        dummy_is_categorical=True,
        consecutive_sequences_are_categorical=False,
        low_unique_int_values_are_categorical=False,
    )
)

### *Correlation Analysis*

In [73]:
correlated_cols = []
if EXECUTE_CORRELATION_ANALYSIS:
    # Get the correlation matrix of the data (excluding the regression and classification targets!)
    # Find the columns that are highly correlated with each other and remove them
    CORRELATION_THRESHOLD = 0.9  # 90% correlation threshold

    df_train_without_clf_target = df_train.drop(columns=[CLF_TARGET])
    correlated_cols, corr, summary = st.correlated_columns(
        df_train_without_clf_target, threshold=CORRELATION_THRESHOLD
    )

    print(f"\nFound {len(correlated_cols)} cols with correlation >= {CORRELATION_THRESHOLD}")
    print(correlated_cols)

    if not NO_PLOTS:
        with plt.style.context(PAPER_STYLE):
            plt.figure(figsize=(10, 8), dpi=600)  # Increased figure size
            sns.heatmap(corr, cmap=plt.cm.CMRmap_r, annot=False, xticklabels=False, yticklabels=False)
            plt.savefig("correlation_heatmap.pdf", bbox_inches="tight", format="pdf", dpi=600)
            plt.tight_layout()
            plt.show()

    summary_table = helpers.make_pretty_table(
        summary,
        ["Correlated Column", "Correlated With", "Correlation", "p-value"],
        title="Correlation Summary",
    )
    print(summary_table)

    print(
        f"Number of columns after dropping correlated columns: {df_train.shape[1] - len(correlated_cols)}"
    )

### *One-Hot Encoding of Likely Categorical Features*


In [ ]:
categorical_cols, numerical_cols = helpers.categorical_and_numerical_columns(
    # Drop the target variable and other columns that will be dropped during training
    df_train.drop(columns=[CLF_TARGET] + correlated_cols), 
    dummy_is_categorical=True,
    consecutive_sequences_are_categorical=True,
    low_unique_int_values_are_categorical=True,
)

categorical_encoder, df_train = helpers.handle_categorical_cols(
    df_train, categorical_cols, return_only_encoded=False
)

print(
    helpers.describe_cols(
        df_train,
        dummy_is_categorical=True,
        consecutive_sequences_are_categorical=True,
        low_unique_int_values_are_categorical=True,
    )
)

print(f"Number of numerical columns: {len(numerical_cols)}")
print(f"Number of categorical columns: {len(categorical_cols)}")

### *Remove Correlated Features (again)*

In [75]:
if EXECUTE_CORRELATION_ANALYSIS:
    # Get the correlation matrix of the data (excluding the regression and classification targets!)
    # Find the columns that are highly correlated with each other and remove them
    CORRELATION_THRESHOLD = 0.9  # 90% correlation threshold

    df_train_without_clf_target = df_train.drop(columns=[CLF_TARGET] + correlated_cols)
    new_correlated_cols, corr, summary = st.correlated_columns(
        df_train_without_clf_target, threshold=CORRELATION_THRESHOLD
    )

    print(f"\nFound {len(new_correlated_cols)} cols with correlation >= {CORRELATION_THRESHOLD}")
    print(new_correlated_cols)

    if not NO_PLOTS:
        with plt.style.context(PAPER_STYLE):
            plt.figure(figsize=(10, 8), dpi=600)  # Increased figure size
            sns.heatmap(corr, cmap=plt.cm.CMRmap_r, annot=False, xticklabels=False, yticklabels=False)
            plt.savefig("correlation_heatmap_after.pdf", bbox_inches="tight", format="pdf", dpi=600)
            plt.tight_layout()
            plt.show()

    summary_table = helpers.make_pretty_table(
        summary,
        ["Correlated Column", "Correlated With", "Correlation", "p-value"],
        title="Correlation Summary",
    )
    print(summary_table)

    correlated_cols += new_correlated_cols
    print(
        f"Number of columns after dropping correlated columns: {df_train.shape[1] - len(new_correlated_cols)}"
    )

In [76]:
# After Feature Selection
if EXECUTE_CORRELATION_ANALYSIS:
    df_train_without_clf_target = df_train.drop(columns=[CLF_TARGET] + correlated_cols)
    _, new_corr, _ = st.correlated_columns(
        df_train_without_clf_target, threshold=CORRELATION_THRESHOLD
    )

    if not NO_PLOTS:
        plt.figure(figsize=(6, 4))
        sns.heatmap(new_corr, cmap=plt.cm.CMRmap_r)
        plt.show()

In [ ]:
if not NO_PLOTS:
    plots.plot_features_vs_target(
        df_train,
        CLF_TARGET,
        figsize=(6, 4),
        features=numerical_cols,
        style=PAPER_STYLE,
        n_features=6,
        max_rows=2,
        is_categorical=True,
        categorical_legend=CLASS_MAPPING,
        x_log_scale=False,
    )

In [ ]:
# Count the number of positive and negative samples
pos_samples = (df_train[CLF_TARGET] == 1).sum()
neg_samples = (df_train[CLF_TARGET] == 0).sum()

if not NO_PLOTS:
    plots.visualize(
        df_train.drop(columns=[CLF_TARGET] + correlated_cols),
        n_components=2,
        method="ica",
        indices=[df_train[CLF_TARGET] == 0, df_train[CLF_TARGET] == 1],
        labels=[
            f"{CLASS_MAPPING[0]}, {neg_samples} samples",
            f"{CLASS_MAPPING[1]}, {pos_samples} samples",
        ],
        style=PAPER_STYLE,
    )

# **Linear classification**

In this part, we will train a simple linear classification model to predict our target `UTILIZATION`.


We will first change our targets (classes: LOW, HIGH) to numeric targets. Then, we solve a logistic regression problem by minimizing the binary cross-entropy function

$$
J(\theta) = -\frac{1}{n} \sum_{i=1}^{n} \left( y_i \log(p_{\theta}(\hat{y}=1 | \mathbf{x}_i)) + (1 - y_i) \log(p_{\theta}(\hat{y}=0 | \mathbf{x}_i)) \right)
$$

where $y_i \in \{0, 1\}$ and $p_{\theta}(\hat{y}=k | \mathbf{x}_i)$ is the probability assigned by our model to class $k$ having observed features $\mathbf{x}_i$.

0 refers to HIGH, and 1 refers to LOW

### Setup

In [79]:
RUNS_DIR = "classification_task_runs"

helpers.ensure_directory_exists(RUNS_DIR)

In [ ]:
# Split into features and target for classification
X = df_train.drop(columns=[CLF_TARGET])
y = df_train[CLF_TARGET]

print(f"Class mapping of column {CLF_TARGET}: {CLASS_MAPPING}\n")
value_counts_dict = y.value_counts().to_dict()

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}", f", value counts in y: {value_counts_dict}")

### Process the data

In [ ]:
X_train, X_test, y_train, y_test = helpers.make_train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

### Fit a model by using training data

In [82]:
if RUN_EXPERIMENTS:
    CLASS_MAPPING = {1: "LOW", 0: "HIGH"}

    # Remove Correlated + Encode (+Likely Categorical)
    # dummy_is_categorical=True,
    # encode_after_remove_correlated=True,
    # consecutive_sequences_are_categorical=True,
    # low_unique_int_values_are_categorical=True,

    # Encode (+Likely Categorical) + Remove Correlated
    # dummy_is_categorical=True,
    # encode_after_remove_correlated=False,
    # consecutive_sequences_are_categorical=True,
    # low_unique_int_values_are_categorical=True,

    # Encode + Remove Correlated
    # dummy_is_categorical=True,
    # encode_after_remove_correlated=False,
    # consecutive_sequences_are_categorical=False,
    # low_unique_int_values_are_categorical=False,

    df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
        experiments.prepare_df_train_pipeline(
            "data/train.csv",
            CLF_TARGET,
            target_is_categorical=True,
            target_categorical_mapping=CLASS_MAPPING,
            remove_cols=[REG_TARGET],
            remove_correlated=False,
            correlation_threshold=CORRELATION_THRESHOLD,
            dummy_is_categorical=True,
            encode_after_remove_correlated=False,
            consecutive_sequences_are_categorical=True,
            low_unique_int_values_are_categorical=True,
            test_size=TEST_SIZE,
            random_state=RANDOM_STATE,
            stratify=True,
            tabs=1,
        )
    )

    df_train_desc = helpers.describe_cols(
        df_train,
        tabs=1,
        dummy_is_categorical=True,
        consecutive_sequences_are_categorical=True,
        low_unique_int_values_are_categorical=True,
    )

    X_train, y_train, X_test, y_test = data

    pipeline = experiments.preprocessing_pipeline2(
        drop_columns=correlated_cols, numerical_scaler=StandardScaler()
    )
    pipeline = experiments.extend_pipeline(
        pipeline,
        ("classifier", None),
    )

    # Define the parameter grid
    base_param_grid = {
        # Classifiers
        "classifier": [
            LogisticRegression(),
        ],
    }

    # Make a list of param grids by combining the preprocessing and classifier hyperparameters
    param_grids = construct_param_grids_list(base_param_grid, key="classifier")

    # Iterate over each classifier
    tuning.run_model_search(
        X_train,
        y_train,
        X_test,
        y_test,
        param_grids,
        pipeline,
        task_type="classification",
        refit_classification="f1-score",
        output_dir=RUNS_DIR,
        run_dir_suffix="Encode (+Likely Categorical)",
        additional_save_on_run={
            "categorical_encoder": (categorical_encoder, "pkl"),
            "categorical_cols": (categorical_cols, "pkl"), 
            "numerical_cols": (numerical_cols, "pkl"),
            "correlated_cols": (correlated_cols, "txt"),
            "df_train_desc": (df_train_desc, "txt"),
        },
        use_cuml=USE_CUML,
        subset=1,
        tabs=1,
        verbose=10,
    )

In [ ]:
summary_df = tuning.plot_runs(
    RUNS_DIR,
    ["f1-score"],
    only_runs_with_this_in_name=["LogisticRegression"],
    map_x_axis_with_str="E",
    summary_table=True,
    figsize=(6, 3),
    title=None,
    style=PAPER_STYLE,
    save_path=os.path.join(RUNS_DIR, "encode_correlation_performance_comparison"),
    save_format="pdf",
    xtick_rotation=0,
    xtick_size=8,
    annotate_values=True,
    dpi=600,
    grid=True,
    use_adjust_text=True,
    y_max=0.79
)

### Test Dimensionality Reduction Techniques

In [ ]:
classifier_to_test = "run_20240822_132045_LogisticRegression_Remove Correlated + Encode (+Likely Categorical)"
classifier_to_test_name = tuning.load_run_results(os.path.join(RUNS_DIR, classifier_to_test))["name"]
best_params = tuning.load_from_run(os.path.join(RUNS_DIR, classifier_to_test), "best_params.pkl")

# Preprocess the best_params so that it can be passed to the model(**best_params)
best_params_copy = best_params.copy()
for key in best_params_copy.keys():
    if key.startswith("classifier__"):
        best_params[key.split("__")[1]] = best_params.pop(key)

if "classifier" in best_params:
    del best_params["classifier"]

print(f"Best params for {classifier_to_test_name}:", best_params)

In [85]:
if RUN_EXPERIMENTS:
    CLASS_MAPPING = {1: "LOW", 0: "HIGH"}
    dimensionality_reduction_methods = [
        # FunctionTransformer(),
        # PCA(random_state=RANDOM_STATE),
        # LinearDiscriminantAnalysis(),
        # FastICA(random_state=RANDOM_STATE),
        UMAP(random_state=RANDOM_STATE),
    ]

    for dimensionality_reduction_method in dimensionality_reduction_methods:
        df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
            experiments.prepare_df_train_pipeline(
                "data/train.csv",
                CLF_TARGET,
                target_is_categorical=True,
                target_categorical_mapping=CLASS_MAPPING,
                remove_cols=[REG_TARGET],
                remove_correlated=True,
                correlation_threshold=CORRELATION_THRESHOLD,
                dummy_is_categorical=True,
                encode_after_remove_correlated=True,
                consecutive_sequences_are_categorical=True,
                low_unique_int_values_are_categorical=True,
                test_size=TEST_SIZE,
                random_state=RANDOM_STATE,
                stratify=True,
                tabs=1,
            )
        )

        df_train_desc = helpers.describe_cols(
            df_train,
            tabs=1,
            dummy_is_categorical=True,
            consecutive_sequences_are_categorical=False,
            low_unique_int_values_are_categorical=False,
        )

        X_train, y_train, X_test, y_test = data

        pipeline = experiments.preprocessing_pipeline2(drop_columns=correlated_cols, numerical_scaler=StandardScaler())
        pipeline = experiments.extend_pipeline(
            pipeline,
            ("dimensionality_reduction", dimensionality_reduction_method),
            ("classifier", LogisticRegression(**best_params)),
        )

        # Define the parameter grid
        if not isinstance(dimensionality_reduction_method, FunctionTransformer):
            param_grid = {
                "dimensionality_reduction__n_components": [2, 10, 50, 100, X_train.shape[1]],
                "classifier": [LogisticRegression(**best_params)],
            }
        else:
            param_grid = {"classifier": [LogisticRegression(**best_params)]}

        # For LinearDiscriminantAnalysis, we need to set the number of components to the number of classes - 1
        if isinstance(dimensionality_reduction_method, LinearDiscriminantAnalysis):
            param_grid["dimensionality_reduction__n_components"] = [len(CLASS_MAPPING) - 1]
            param_grid["dimensionality_reduction__solver"] = ["svd", "lsqr", "eigen"]

        if isinstance(dimensionality_reduction_method, UMAP):
            param_grid["dimensionality_reduction__metric"] = ["euclidean", "manhattan", "cosine"]
        
        if isinstance(dimensionality_reduction_method, FastICA):
            param_grid["dimensionality_reduction__whiten"] = ["arbitrary-variance", "unit-variance", False]
            param_grid["dimensionality_reduction__whiten_solver"] = ["eigh", "svd"]

        # Iterate over each classifier
        tuning.run_model_search(
            X_train,
            y_train,
            X_test,
            y_test,
            param_grid,
            pipeline,
            task_type="classification",
            refit_classification="f1-score",
            output_dir=os.path.join(RUNS_DIR, "dimensionality_reduction"),
            run_dir_suffix=dimensionality_reduction_method.__class__.__name__,
            additional_save_on_run={
                "categorical_encoder": (categorical_encoder, "pkl"),
                "categorical_cols": (categorical_cols, "pkl"),
                "numerical_cols": (numerical_cols, "pkl"),
                "correlated_cols": (correlated_cols, "txt"),
                "df_train_desc": (df_train_desc, "txt"),
            },
            use_cuml=USE_CUML,
            tabs=1,
            verbose=10,
        )

In [ ]:
summary_df = tuning.plot_runs(
    os.path.join(RUNS_DIR, "dimensionality_reduction"),
    ["f1-score"],
    runs_names_mapping={
        "FunctionTransformer": "Baseline",
        "PCA": "PCA",
        "LinearDiscriminantAnalysis": "LDA",
        "FastICA": "ICA",
        "UMAP": "UMAP",
    },
    summary_table=True,
    figsize=(6, 3),
    style=PAPER_STYLE,
    title=None,
    save_path=os.path.join(RUNS_DIR, "dimensionality_reduction", "performance_comparison"),
    save_format="pdf",
    dpi=600,
    xtick_rotation=0,
    xtick_size=6,
    annotate_values=True,
    grid=True,
    use_adjust_text=True,
    y_max=1
)

In [ ]:
RUN_EXPERIMENTS = True
if RUN_EXPERIMENTS:
    CLASS_MAPPING = {1: "LOW", 0: "HIGH"}
    skew_transformer_methods = [
        FunctionTransformer(),
        PowerTransformer(method="yeo-johnson"),
        QuantileTransformer(output_distribution="normal"),
        QuantileTransformer(output_distribution="uniform"),
        Normalizer(),
    ]

    for skew_transformer_method in skew_transformer_methods:
        df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
            experiments.prepare_df_train_pipeline(
                "data/train.csv",
                CLF_TARGET,
                target_is_categorical=True,
                target_categorical_mapping=CLASS_MAPPING,
                remove_cols=[REG_TARGET],
                remove_correlated=True,
                correlation_threshold=CORRELATION_THRESHOLD,
                dummy_is_categorical=True,
                encode_after_remove_correlated=True,
                consecutive_sequences_are_categorical=True,
                low_unique_int_values_are_categorical=True,
                test_size=TEST_SIZE,
                random_state=RANDOM_STATE,
                stratify=True,
                tabs=1,
            )
        )

        df_train_desc = helpers.describe_cols(
            df_train,
            tabs=1,
            dummy_is_categorical=True,
            consecutive_sequences_are_categorical=True,
            low_unique_int_values_are_categorical=True,
        )

        X_train, y_train, X_test, y_test = data

        pipeline = experiments.preprocessing_pipeline2(drop_columns=correlated_cols, skew_transformer=skew_transformer_method)
        pipeline = experiments.extend_pipeline(
            pipeline,
            ("classifier", LogisticRegression(**best_params)),
        )

        # Define the parameter grid
        param_grid = {"classifier": [LogisticRegression(**best_params)]}

        run_dir_suffix = skew_transformer_method.__class__.__name__

        if isinstance(skew_transformer_method, QuantileTransformer):
            run_dir_suffix += f"_{skew_transformer_method.output_distribution}"

        # Iterate over each classifier
        tuning.run_model_search(
            X_train,
            y_train,
            X_test,
            y_test,
            param_grid,
            pipeline,
            task_type="classification",
            refit_classification="f1-score",
            output_dir=os.path.join(RUNS_DIR, "skew_transformers"),
            run_dir_suffix=run_dir_suffix,
            additional_save_on_run={
                "categorical_encoder": (categorical_encoder, "pkl"),
                "categorical_cols": (categorical_cols, "pkl"),
                "numerical_cols": (numerical_cols, "pkl"),
                "correlated_cols": (correlated_cols, "txt"),
                "df_train_desc": (df_train_desc, "txt"),
            },
            use_cuml=USE_CUML,
            tabs=1,
            verbose=10,
        )

In [ ]:
summary_df = tuning.plot_runs(
    os.path.join(RUNS_DIR, "skew_transformers"),
    ["f1-score"],
    runs_names_mapping={
        "FunctionTransformer": "Baseline",
        "PowerTransformer": "Yeo-Johnson",
        "QuantileTransformer_normal": "Quantile (Normal)",
        "QuantileTransformer_uniform": "Quantile (Uniform)",
        "Normalizer": "Normalized",
    },
    summary_table=True,
    figsize=(6, 3),
    style=PAPER_STYLE,
    title=None,
    save_path=os.path.join(RUNS_DIR, "skew_transformers", "performance_comparison"),
    save_format="pdf",
    dpi=600,
    xtick_rotation=0,
    xtick_size=6,
    annotate_values=True,
    grid=True,
    use_adjust_text=True,
    y_max=0.79
)

In [ ]:
RUN_EXPERIMENTS = True
if RUN_EXPERIMENTS:
    CLASS_MAPPING = {1: "LOW", 0: "HIGH"}
    scalers = [
        FunctionTransformer(),
        StandardScaler(),
        RobustScaler(),
        MinMaxScaler(),
    ]

    for scaler in scalers:
        df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
            experiments.prepare_df_train_pipeline(
                "data/train.csv",
                CLF_TARGET,
                target_is_categorical=True,
                target_categorical_mapping=CLASS_MAPPING,
                remove_cols=[REG_TARGET],
                remove_correlated=True,
                correlation_threshold=CORRELATION_THRESHOLD,
                dummy_is_categorical=True,
                encode_after_remove_correlated=True,
                consecutive_sequences_are_categorical=True,
                low_unique_int_values_are_categorical=True,
                test_size=TEST_SIZE,
                random_state=RANDOM_STATE,
                stratify=True,
                tabs=1,
            )
        )

        df_train_desc = helpers.describe_cols(
            df_train,
            tabs=1,
            dummy_is_categorical=True,
            consecutive_sequences_are_categorical=True,
            low_unique_int_values_are_categorical=True,
        )

        X_train, y_train, X_test, y_test = data

        pipeline = experiments.preprocessing_pipeline2(drop_columns=correlated_cols, numerical_scaler=scaler)
        pipeline = experiments.extend_pipeline(
            pipeline,
            ("classifier", LogisticRegression(**best_params)),
        )

        # Define the parameter grid
        param_grid = {"classifier": [LogisticRegression(**best_params)]}

        run_dir_suffix = scaler.__class__.__name__

        # Iterate over each classifier
        tuning.run_model_search(
            X_train,
            y_train,
            X_test,
            y_test,
            param_grid,
            pipeline,
            task_type="classification",
            refit_classification="f1-score",
            output_dir=os.path.join(RUNS_DIR, "scalers"),
            run_dir_suffix=run_dir_suffix,
            additional_save_on_run={
                "categorical_encoder": (categorical_encoder, "pkl"),
                "categorical_cols": (categorical_cols, "pkl"),
                "numerical_cols": (numerical_cols, "pkl"),
                "correlated_cols": (correlated_cols, "txt"),
                "df_train_desc": (df_train_desc, "txt"),
            },
            use_cuml=USE_CUML,
            tabs=1,
            verbose=10,
        )

In [ ]:
summary_df = tuning.plot_runs(
    os.path.join(RUNS_DIR, "scalers"),
    ["f1-score"],
    runs_names_mapping={
        "FunctionTransformer": "Baseline",
        "StandardScaler": "StandardScaler",
        "RobustScaler": "RobustScaler",
        "MinMaxScaler": "MinMaxScaler"
    },
    summary_table=True,
    figsize=(6, 3),
    style=PAPER_STYLE,
    title=None,
    save_path=os.path.join(RUNS_DIR, "scalers", "performance_comparison"),
    save_format="pdf",
    dpi=600,
    xtick_rotation=0,
    xtick_size=6,
    annotate_values=True,
    grid=True,
    legend_loc="upper right",
    use_adjust_text=True,
    y_max=0.79
)

In [ ]:
df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
    experiments.prepare_df_train_pipeline(
        "data/train.csv",
        CLF_TARGET,
        target_is_categorical=True,
        target_categorical_mapping=CLASS_MAPPING,
        remove_cols=[REG_TARGET],
        remove_correlated=True,
        correlation_threshold=CORRELATION_THRESHOLD,
        dummy_is_categorical=True,
        encode_after_remove_correlated=True,
        consecutive_sequences_are_categorical=True,
        low_unique_int_values_are_categorical=True,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        stratify=True,
        tabs=1,
    )
)

pipeline = experiments.preprocessing_pipeline2(drop_columns=correlated_cols, numerical_scaler=RobustScaler())

X_train, y_train, X_test, y_test = data

X_train = pipeline.fit_transform(X_train, y_train)
X_test = pipeline.transform(X_test)

cv = StratifiedKFold(5)

best_params = tuning.load_from_run(
    os.path.join(RUNS_DIR, "final_classifiers_with_select_k_best", "run_20240823_205939_GradientBoostingClassifier_"),
    "best_params.pkl",
)
# Preprocess the best_params so that it can be passed to the model(**best_params)
best_params_copy = best_params.copy()
for key in best_params_copy.keys():
    if key.startswith("classifier__"):
        best_params[key.split("__")[1]] = best_params.pop(key)

if "classifier" in best_params:
    del best_params["classifier"]
if "select__k" in best_params:
    del best_params["select__k"]
    del best_params["select__score_func"]

print(f"Best params:", best_params)

clf = GradientBoostingClassifier(**best_params)
rfecv = RFECV(estimator=clf, step=50, cv=cv, scoring="f1_macro", min_features_to_select=1, n_jobs=-2)

rfecv.fit(X_train, y_train)

print(f"Optimal number of features: {rfecv.n_features_}")

In [ ]:
cv_results = pd.DataFrame(rfecv.cv_results_)

with plt.style.context(PAPER_STYLE):
    plt.figure(figsize=(6, 4), dpi=600)

    # Plot error bars
    plt.errorbar(
        x=cv_results["n_features"],
        y=cv_results["mean_test_score"],
        yerr=cv_results["std_test_score"],
        fmt="o-",  # Circle markers connected by lines
        color="black",
        ecolor="black",
        elinewidth=0.5,
        capsize=2.5,
        capthick=0.5,
        markersize=4,
        linewidth=1,
    )

    # Set labels and title
    plt.xlabel("Number of features selected", fontweight="bold")
    plt.ylabel("Mean Validation F1-score", fontweight="bold")

    # Set tick parameters
    plt.tick_params(axis="both", which="major", labelsize=10)

    plt.savefig(os.path.join(RUNS_DIR, "feature_selection_performance.pdf"), bbox_inches="tight", format="pdf", dpi=600)
    plt.tight_layout()
    plt.show()

In [ ]:
RUN_EXPERIMENTS = True
if RUN_EXPERIMENTS:
    CLASS_MAPPING = {1: "LOW", 0: "HIGH"}

    df_train, categorical_encoder, categorical_cols, numerical_cols, correlated_cols, data = (
        experiments.prepare_df_train_pipeline(
            "data/train.csv",
            CLF_TARGET,
            target_is_categorical=True,
            target_categorical_mapping=CLASS_MAPPING,
            remove_cols=[REG_TARGET],
            remove_correlated=True,
            correlation_threshold=CORRELATION_THRESHOLD,
            dummy_is_categorical=True,
            encode_after_remove_correlated=True,
            consecutive_sequences_are_categorical=True,
            low_unique_int_values_are_categorical=True,
            test_size=TEST_SIZE,
            random_state=RANDOM_STATE,
            stratify=True,
            tabs=1,
        )
    )

    pipeline = experiments.preprocessing_pipeline2(drop_columns=correlated_cols, numerical_scaler=RobustScaler())

    pipeline = experiments.extend_pipeline(
        pipeline,
        ("select", SelectKBest()),
        ("nystroem", FunctionTransformer()),  # For kernel approximation (LinearSVC, NuSVC)
        ("classifier", None),
    )

    df_train_desc = helpers.describe_cols(
        df_train,
        tabs=1,
        dummy_is_categorical=True,
        consecutive_sequences_are_categorical=True,
        low_unique_int_values_are_categorical=True,
    )
    
    # Define the parameter grid
    base_param_grid = {
        "select__k": [70],
        "select__score_func": [mutual_info_classif],
        # Classifiers
        "classifier": [
#            GaussianNB(),
            LogisticRegression(),
            # KNeighborsClassifier(),
            # LinearDiscriminantAnalysis(),
            # QuadraticDiscriminantAnalysis(),
            # RidgeClassifier(),
            # PassiveAggressiveClassifier(),
            # SGDClassifier(),
            # GradientBoostingClassifier(),
            # XGBClassifier(),
            # LinearSVC(),
            # NuSVC(),
        ],
        "nystroem": [Nystroem()],
        "nystroem__n_components": [100, 500, 1000],
        "nystroem__kernel": ['rbf', 'poly', 'sigmoid'],
    }

    # Make a list of param grids by combining the preprocessing and classifier hyperparameters
    param_grids = construct_param_grids_list(base_param_grid, key="classifier")

    X_train, y_train, X_test, y_test = data

    # Make a list of param grids by combining the preprocessing and classifier hyperparameters
    param_grids = construct_param_grids_list(base_param_grid, key="classifier")

    # Iterate over each classifier
    tuning.run_model_search(
        X_train,
        y_train,
        X_test,
        y_test,
        param_grids,
        pipeline,
        task_type="classification",
        refit_classification="f1-score",
        output_dir=os.path.join(RUNS_DIR, "final_classifiers"),
        additional_save_on_run={
            "categorical_encoder": (categorical_encoder, "pkl"),
            "categorical_cols": (categorical_cols, "pkl"), 
            "numerical_cols": (numerical_cols, "pkl"),
            "correlated_cols": (correlated_cols, "txt"),
            "df_train_desc": (df_train_desc, "txt"),
        },
        use_cuml=USE_CUML,
        subset=2,
        tabs=1,
        verbose=10,
    )

In [ ]:
summary_df = tuning.plot_runs(
    os.path.join(RUNS_DIR, "final_classifiers_mutual_info_classif"),
    ["f1-score"],
    sort_by="f1-score",
    runs_names_mapping={
        "GradientBoostingClassifier": "GBC",
        "LinearDiscriminantAnalysis": "LDA",
        "QuadraticDiscriminantAnalysis": "QDA",
        "PassiveAggressiveClassifier": "PAC",
    },
    summary_table=True,
    figsize=(6, 3),
    style=PAPER_STYLE,
    title=None,
    save_path=os.path.join(RUNS_DIR, "final_classifiers_mutual_info_classif", "model_performance_comparison"),
    save_format="pdf",
    dpi=600,
    xtick_rotation=30,
    xtick_size=6,
    annotate_values=True,
    grid=True,
    use_adjust_text=True,
    y_max=1
)

Now evaluate your model. Check the appendix for details on micro, macro and weighted averaging

In [ ]:
best_run = "run_20240823_215649_XGBClassifier_"
best_run_path = os.path.join(RUNS_DIR, "final_classifiers_mutual_info_classif", best_run)
best_classifier = tuning.load_run_model(best_run_path)
best_classifier_name = best_run.split("_")[-2]
categorical_encoder = tuning.load_from_run(best_run_path, "categorical_encoder")
categorical_cols = tuning.load_from_run(best_run_path, "categorical_cols")
X_train, y_train, X_test, y_test = tuning.load_from_run(best_run_path, "data")

In [ ]:
datasets = {"training data": [X_train, y_train], "validation data": [X_test, y_test]}

for split_name, dataset in datasets.items():
    X_i, y_i = dataset
    y_pred = best_classifier.predict(X_i)
    print(f"\nSplit: {split_name}")

    print(skm.classification_report(y_i, y_pred))

In [ ]:
X_transformed = best_classifier[:-1].transform(X_train)

if not NO_PLOTS:
    plots.visualize(
        X_transformed,
        n_components=3,
        method="pca",
        indices=[y_train == 0, y_train == 1],
        labels=[
            f"{CLASS_MAPPING[0]}, {len(y_train[y_train == 0])} samples",
            f"{CLASS_MAPPING[1]}, {len(y_train[y_train == 1])} samples",
        ],
        style=PAPER_STYLE,
    )

In [ ]:
plots.plot_pr_roc_curves(
    best_classifier,
    X_test,
    y_test,
    clf_name=best_classifier_name,
    style=PAPER_STYLE,
)

In [ ]:
print(
    f"Original f1-score: {scorers.clf_scorers['f1_macro'](best_classifier, X_test, y_test)}"
)

tuned_model = TunedThresholdClassifierCV(
    estimator=best_classifier,
    scoring=scorers.clf_scorers["f1_macro"],
    store_cv_results=True,
)
tuned_model.fit(X_train, y_train)
print(f"{tuned_model.best_threshold_ = :0.2f}")
print(
    f"After tuning f1-score: {scorers.clf_scorers['f1_macro'](tuned_model, X_test, y_test)}"
)

In [ ]:
# Comparison of the cut-off point for the vanilla and tuned model
plots.plot_pr_roc_curves(
    best_classifier,
    X_test,
    y_test,
    tuned_model,
    clf_name=best_classifier_name,
    show_objective_score=False,
    style=PAPER_STYLE,
)

In [ ]:
datasets = {"training data": [X_train, y_train], "validation data": [X_test, y_test]}

for split_name, dataset in datasets.items():
    X_i, y_i = dataset
    y_pred = tuned_model.predict(X_i)
    print(f"\nSplit: {split_name}")

    print(skm.classification_report(y_i, y_pred))

At this point, we can use our model to predict healthcare utilization on the test set.

We again need to follow a specific namim format when saving the predictions. Similarly to before, the name of the file should be `<TEAM_ID>__<SPLIT>__clf_pred.npy`.



In [127]:
best_classifier = tuned_model

In [ ]:
# Run this to save a file with your predictions on the test set to be submitted
split = "test_public"  # replace by 'test_private' for FINAL submission

df_test = pd.read_csv(f"data/{split}.csv")

# Process data
# Make sure that we keep only the categorical cols that exist here
categorical_cols = helpers.remove_non_existent_columns(categorical_cols, df_test.columns)

# Handle the categorical columns in the test set
df_test = helpers.encode_categorical_cols(
    df_test, categorical_cols, categorical_encoder, return_only_encoded=False
)

y_hat = best_classifier.predict(df_test)

# Save the results with the format <TEAM_ID>__<SPLIT>__clf_pred.npy

folder = "./results"
np.save(os.path.join(folder, f"{team_id}__{split}__clf_pred.npy"), y_hat)

# Submission to CMS

Put your .npy files for both regression and classification tasks in the same zip file. Please name the file as `<TEAM_ID>.zip` (e.g. `123.zip`) and upload it to CMS system. It is essential that the files inside the .zip are named as follow:

`<TEAM_ID>__<SPLIT>__reg_pred.npy` \
`<TEAM_ID>__<SPLIT>__clf_pred.npy`

Above, `<SPLIT>` should correspond to `test_public` for the leaderboard and `test_private` for the final submission.
As long as the `test_private.csv` data file is not released yet, the zip will contain only two files.


In [ ]:
# Run this to save a file with your predictions on the test set to be submitted
split = "test_private"  # replace by 'test_private' for FINAL submission

df_test = pd.read_csv(f"data/{split}.csv")

# Process data
# Make sure that we keep only the categorical cols that exist here
categorical_cols = helpers.remove_non_existent_columns(categorical_cols, df_test.columns)

# Handle the categorical columns in the test set
df_test = helpers.encode_categorical_cols(
    df_test, categorical_cols, categorical_encoder, return_only_encoded=False
)

y_hat = best_classifier.predict(df_test)

# Save the results with the format <TEAM_ID>__<SPLIT>__clf_pred.npy

folder = "./results/test_private"
helpers.ensure_directory_exists(folder)
np.save(os.path.join(folder, f"{team_id}__{split}__clf_pred.npy"), y_hat)

### Appendix: Reminders about macro and micro averaging:

When evaluating a classification model using `skm.classification_report(y_i, y_pred)` as done above, we get a macro and a weighted average.

In the context of computing F1-score, "macro" and "micro" averaging are two commonly used techniques to aggregate the per-class F1-scores.

**Micro-average**: Compute the F1-score globally by counting the total true positives, false negatives, and false positives over all classes, and then calculating precision, recall, and F1-score using these aggregated values.

**Macro-average**: Calculate the F1-score for each class separately, and then take the average of these per-class F1-scores.

The main difference between these two techniques is the way they treat class imbalance. Micro-average treats all classes equally, regardless of their size, while macro-average treats each class equally, regardless of the number of samples in that class.

Micro-average is often used when we care about overall performance across all classes, and we want to give more weight to the performance on larger classes. In contrast, macro-average is often used when we want to evaluate the performance on each class separately and give equal weight to each class.


In addition to micro and macro averaging, there is another common technique for computing the F1-score called **weighted averaging**.

**Weighted averaging** is similar to macro averaging in that it computes the per-class F1-score and then takes the average of these scores. However, unlike macro averaging, weighted averaging takes into account the number of samples in each class when computing the average. Specifically, the weighted average is computed as follows:

- Compute the F1-score for each class separately.
- Compute the weight for each class as the number of samples in that class divided by the total number of samples.
- Compute the weighted average of the per-class F1-scores, where each per-class F1-score is weighted by the weight of that class.

The weighted average is commonly used when the dataset is imbalanced, meaning that some classes have many more samples than others. In such cases, using the simple average (macro-average) would give too much weight to the smaller classes, while using micro-average would give too much weight to the larger classes. The weighted average strikes a balance between these two approaches by giving more weight to the classes with more samples while still taking into account the performance of all classes.


When computing the F1 score for the leaderboard and the final challenge results, we will be using the macro averaging strategy.